In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.impute import SimpleImputer #fill missing values with mean, meadian and mode
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder

In [5]:
df = pd.read_csv('covid_toy.csv')

In [6]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [9]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2, random_state=2)

### 1. General Approach (Aam Zindagi)

In [27]:
# filling missing values using simple imputer in fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [42]:
# OrdinalEncoding -> Cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# Also the test data
X_test_cough = oe.transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [51]:
# OneHotEncoding -> gender and city
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

#ALso the test data 
X_test_gender_city = ohe.transform(X_test[['gender', 'city']])


In [56]:
# EXtracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

#Also the test data
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

In [60]:
X_train_transformed = np.concatenate((X_train_age, X_train_gender_city, X_train_fever,X_train_cough), axis=1)

#also the test data
X_test_transformed = np.concatenate((X_test_age, X_test_gender_city, X_test_fever, X_test_cough), axis=1)
X_train_transformed.shape

(80, 7)

### 2. Smart Approach using Column transform (Mentos Zindagi)

In [61]:
from sklearn.compose import ColumnTransformer

In [65]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])

], remainder='passthrough')

In [66]:
transformer.fit_transform(X_train).shape

(80, 7)

In [67]:
transformer.transform(X_test).shape

(20, 7)